# Santander Dev Week 2023 (ETL com Python)

**Contexto:** Você é um cientista de dados no Santander e recebeu a tarefa de envolver seus clientes de maneira mais personalizada. Seu objetivo é usar o poder da IA Generativa para criar mensagens de marketing personalizadas que serão entregues a cada cliente.

**Condições do Problema:**

1. Você recebeu uma planilha simples, em formato CSV ('SDW2023.csv'), com uma lista de IDs de usuário do banco:
  ```
  UserID
  1
  2
  3
  4
  5
  ```
2. Seu trabalho é consumir o endpoint `GET https://sdw-2023-prd.up.railway.app/users/{id}` (API da Santander Dev Week 2023) para obter os dados de cada cliente.
3. Depois de obter os dados dos clientes, você vai usar a API do ChatGPT (OpenAI) para gerar uma mensagem de marketing personalizada para cada cliente. Essa mensagem deve enfatizar a importância dos investimentos.
4. Uma vez que a mensagem para cada cliente esteja pronta, você vai enviar essas informações de volta para a API, atualizando a lista de "news" de cada usuário usando o endpoint `PUT https://sdw-2023-prd.up.railway.app/users/{id}`.



In [1]:
# Utilize sua própria URL se quiser ;)
# Repositório da API: https://github.com/Caique-HPF/santander-dev-week-2023-api.git
sdw2023_api_url = 'https://sdw-2023-prd.up.railway.app'

## **E**xtract

Extraia a lista de IDs de usuário a partir do arquivo CSV. Para cada ID, faça uma requisição GET para obter os dados do usuário correspondente.

In [2]:
import pandas as pd

df = pd.read_csv('SDW2023.csv')
user_ids = df['UserID'].tolist()
print(user_ids)

[3379, 3380, 3381]


In [3]:
import requests
import json

def get_user(id):
  response = requests.get(f'{sdw2023_api_url}/users/{id}')
  return response.json() if response.status_code == 200 else None

users = [user for id in user_ids if (user := get_user(id)) is not None]
print(json.dumps(users, indent=2))

[
  {
    "id": 3379,
    "name": "Caique",
    "account": {
      "id": 3594,
      "number": "54789-1",
      "agency": "0020",
      "balance": 0.0,
      "limit": 500.0
    },
    "card": {
      "id": 3275,
      "number": "4321 1234 1234 1111",
      "limit": 1000.0
    },
    "features": [],
    "news": []
  },
  {
    "id": 3380,
    "name": "Kerolin",
    "account": {
      "id": 3595,
      "number": "67542-1",
      "agency": "0020",
      "balance": 0.0,
      "limit": 500.0
    },
    "card": {
      "id": 3276,
      "number": "4321 1234 1234 2222",
      "limit": 1000.0
    },
    "features": [],
    "news": []
  },
  {
    "id": 3381,
    "name": "Denner",
    "account": {
      "id": 3596,
      "number": "67545-1",
      "agency": "0020",
      "balance": 0.0,
      "limit": 500.0
    },
    "card": {
      "id": 3277,
      "number": "4321 1234 1234 3333",
      "limit": 1000.0
    },
    "features": [],
    "news": []
  }
]


## **T**ransform

Foi utilizado um arquivo em TXT gerando mensagens automáticas


In [17]:
# como não tenho mais token no ChatGPT, criei um arquivo para as mensagens automática

import random

with open ('textoAutomaticos.txt', 'r') as documento:
  mensagens = documento.readlines()


In [18]:
from IPython.utils import text
def generate_ai_news(user):

  mensagem = random.choice(mensagens).strip()
  text = (f"Olá {user['name']}, {mensagem}")
  return text

for user in users:
  news = generate_ai_news(user)
  print(news)
  user['news'].append({
      "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit.svg",
      "description": news
  })

Olá Caique, O futuro é incerto, mas seus investimentos podem torná-lo mais seguro. Comece a investir hoje para construir um amanhã mais sólido!
Olá Kerolin, Apenas economizar não é suficiente para alcançar seus objetivos financeiros. Investir é o caminho para fazê-los crescer.
Olá Denner, Lembre-se, o tempo está do seu lado quando se trata de investir. Quanto mais cedo você começar, mais tempo seu dinheiro terá para crescer. Não hesite!


## **L**oad

Atualize a lista de "news" de cada usuário na API com a nova mensagem gerada.

In [19]:
def update_user(user):
  response = requests.put(f"{sdw2023_api_url}/users/{user['id']}", json=user)
  return True if response.status_code == 200 else False

for user in users:
  success = update_user(user)
  print(f"User {user['name']} updated? {success}!")

User Caique updated? True!
User Kerolin updated? True!
User Denner updated? True!
